In [1]:
from search_helpers import *
from parser import *
import ipaddress as ipa

# NetworkConfigParser

A small module to parse structured documents, like Cisco or Juniper network device configurations. Maintains
relationships among lines for ease of further parsing and analysis. Parses IP addresses for easier matching using the
ipaddress library.

## Quick Start and Examples

---

### Example 1: Find references to an interface name

1. We will read in the example configuration shown below, stored in the `config` variable as a string. The `parse_from_str()` function will parse this string, returning a list of `DocumentLine()` objects.

In [2]:
config = """interface TenGigE0/1/0/1
 description Backbone Circuit to North Pudsey
 cdp
 mtu 2060
 ipv4 address 192.0.2.101 255.255.255.252
 load-interval 30
!
interface Loopback10
 description Router ID
 ipv4 address 192.0.2.1 255.255.255.255
!
router isis coreIGP
 net 49.0000.1920.0000.2001.00
 log adjacency changes
 address-family ipv4 unicast
  metric-style wide
  mpls traffic-eng level-1-2
  mpls traffic-eng router-id Loopback10
 !
 interface Loopback10
  passive
  circuit-type level-2-only
  address-family ipv4 unicast
  !
 !
 interface TenGigE0/1/0/1
  circuit-type level-2-only
  point-to-point
  address-family ipv4 unicast
   metric 1000
   mpls ldp sync
!
rsvp
 interface TenGigE0/1/0/1
 !
 interface TenGigE0/0/0/0
 !
!
mpls traffic-eng
 interface TenGigE0/0/0/0
 !
 interface TenGigE0/1/0/1
 !
!
mpls ldp
 !
 igp sync delay on-session-up 10
 router-id 192.0.2.1
 !
 session protection
 !
 interface TenGigE0/0/0/0
 !
 interface TenGigE0/1/0/1
 !
"""

doc_lines = parse_from_str(config)

doc_lines[0:6]

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=2: " description Backbone Circuit to North Pudsey">,
 <DocumentLine gen=2 num_children=0 line_num=3: " cdp">,
 <DocumentLine gen=2 num_children=0 line_num=4: " mtu 2060">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">,
 <DocumentLine gen=2 num_children=0 line_num=6: " load-interval 30">]

2. Next, let's identify lines pertaining to TenGigE0/1/0/1. The `find_lines_regex()` function helps us do this.

We supply the list created in the step above, plus a lambda function to identify the lines we want matched.

By default, only the matched lines are returned. Parent lines are not included by default, but in this case we will want to see the section in which the interface appears. So we add `include_ancestors=True` to get the interface name.

In [7]:
intf_lines = find_lines_regex(doc_lines, r'TenGigE0/1/0/1', include_ancestors=True)


3. The filtered list contains the matched interface lines with all section header lines ("router isis", "mpls ldp", etc.) leading to those matches.


In [8]:
intf_lines

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">]

Converting these items to strings works as the user might expect. Simply use a list comprehension to convert the items to str.

In [6]:
[str(i) for i in intf_lines]

['interface TenGigE0/1/0/1',
 'router isis coreIGP',
 ' interface TenGigE0/1/0/1',
 'rsvp',
 ' interface TenGigE0/1/0/1',
 'mpls traffic-eng',
 ' interface TenGigE0/1/0/1',
 'mpls ldp',
 ' interface TenGigE0/1/0/1']

---

## Example:

---

### Example 2: Find objects matching an IP network

Continuing with our example above, we want to find a particular IP network in the configuration. The line object, DocumentLine, has a handy method called `has_ip()`:

In [38]:
ip = ipa.ip_network('192.0.2.100/30')
find_lines(doc_lines, lambda o: o.has_ip(ip), include_ancestors=True)

[<DocumentLine gen=1 num_children=5 line_num=1: "interface TenGigE0/1/0/1">,
 <DocumentLine gen=2 num_children=0 line_num=5: " ipv4 address 192.0.2.101 255.255.255.252">]

---

### Example 3: Extracting all IP addresses and networks referenced in a configuration

IPs are easy to gather with a set comprehension (shown below) or a list comprehension.

In [36]:
all_ip_addrs = {j for i in doc_lines for j in i.ip_addrs if not i.is_comment}
all_ip_addrs

{IPv4Address('192.0.2.1'), IPv4Address('192.0.2.101')}

In [37]:
all_ip_networks = {j for i in doc_lines for j in i.ip_nets if not i.is_comment}
all_ip_networks

{IPv4Network('192.0.2.1/32'), IPv4Network('192.0.2.100/30')}

---

### Example 5: Find route policies and their contents

Find lines with the "route-policy" term in them, including only the ancestor lines:

```
rp_lines = find_line_ancestors_descendants(doc_lines, lambda o: o.startswith('route-policy') or o.startswith('route-map'))
```

The list `rp_lines` now may contain something similar to:

```
route-map BGP-TO-OSPF deny 10
 match ip address prefix-list BLOCK-PREFIX
route-map BGP-TO-OSPF deny 20
 set metric 100
```

Or for an IOS XR device:

```
route-policy BGP-TO-OSPF
 if destination not in BLOCK-PREFIX then
  set metric 100
 else
  drop
 endif
end-policy
```

---

### Example 6: Find IOS XR BGP neighbors that do not have a particular route-policy configured

Here we include descendants only.

```
doc_lines = parse_from_file('config-filename.conf')
router_bgp_lines = find_line(doc_lines, lambda o: o.startswith('router bgp '))
nbr_lines = find_line_and_descendants(router_bgp_lines, lambda o: o.lstrip().startswith('neighbor ') and not
  any(['route-policy OSPF-TO-BGP' in i for i in o.all_descendants])
```

---

### Example 7: Find top-level statements only

```
doc_lines = parse_from_file('config-filename.conf')
top_level_lines = [i for i in doc_lines if i.gen == 1]
```

---
